## Reactive Functions and Expressions

:::{note}
This guide focuses on the reactive functionalities introduced in Param 2.0, which are **experimental** but ready for [feedback](https://github.com/holoviz/param/issues/new/choose)!
:::

In Param, multiple paradigms for dynamic behavior coexist. The [Dependencies and Watchers](Dependencies_and_Watchers.ipynb) guide delves into an imperative 'push' model, where explicit callbacks handle Parameter updates, often suited for GUI environments where user interactions drive state changes.

Param 2.0 introduces a declarative, reactive model. Reactive expressions automatically update when their referenced Parameters change. This model encourages you to specify 'what' should happen, letting Param manage 'how,' thereby simplifying code logic and enhancing modularity. Unlike the 'push' model, which may require complex event orchestration, the reactive model emphasizes high-level relationships. This makes it versatile, scaling from simple to complex use cases. For instance, the reactive model is a good fit in data transformation pipelines and real-time dashboards, automatically updating dependent steps or visualizations when underlying Parameters change. The reactive approach allows you to focus on defining the transformation or relationship logic without worrying about the sequence of updates.

This guide covers two main approaches to the reactive model: 
1. **Reactive Expressions:** With `.rx`, create reactive proxies for Parameters or objects, which recompute as inputs change.
2. **Reactive Functions:** Using `.bind`, auto-invoked functions update when their inputs change, offering a more declarative alternative to `.watch()`.

:::{note} The code in this guide is designed to be run incrementally to observe the behavior of reactive expressions. If you're reading a rendered version online, keep in mind that the entire page will have been executed, affecting the output of earlier lines. To fully experience the live updates, download this page as a Jupyter Notebook and run through it line by line.

## Getting Started

Param's `rx` feature allows you to create reactive values and expressions, enabling immediate updates to your results as you interactively modify values, avoiding the need for explicit callbacks or managing state manually.

Before we dive in to discover how `rx` works behind the scenes, let's get started with a concrete example of loading some data into a [Pandas](https://pandas.pydata.org) DataFrame and then displaying it:

In [ ]:
import pandas as pd
import param

from param import rx

In [ ]:
URL = 'https://datasets.holoviz.org/penguins/v1/penguins.csv'
nrows = rx(2)
df = rx(pd.read_csv(URL))
df.head(nrows)

Here, this is just the same code you'd normally use to make a DataFrame, except for `rx()` being used to mark `nrows` and `df` as being reactive. As you can see, the reactive DataFrame works like any other DataFrame, using `.head()` and any other DataFrame methods as usual. But now, let's see what happens if we update the value of `nrows`:

In [ ]:
nrows.rx.value += 2

Whoa! As long as you are running a Jupyter notebook with a live Python process, you should have seen the dataframe "head" output _in_ _the_ _previous_ _cell_ update to the new value of `nrows`.  That's because the reactive `df` expression being displayed in that cell captures the full pipeline of operations, automatically re-running `head` because the `nrows` has now changed. 

We've done this without having to write any special callbacks or any new functions, instead using special Python objects that capture the operations you've invoked and replay them as needed when inputs change.

These updates should happen immediately (not only when the code cell finishes executing):

In [ ]:
import time

for i in range(5,10):
    nrows.rx.value = i
    time.sleep(1)

You should see the previous `df.head` output react to each time `nrows` is changed, updating to reflect the current state.

Next, let's explore a more intricate example. Although it involves a more complex pipeline, the code remains similar to what you'd write for a non-reactive Pandas DataFrame. To confirm, you can simply remove the `rx` calls:

In [ ]:
import numpy as np

style = rx('color: white; background-color: {color}')
color = rx('darkblue')

def highlight_max(s, props=''):
    if s.dtype.kind not in 'f':
        return np.full_like(s, False)
    return np.where(s == np.nanmax(s.values), props, '')

styled_df = df.head(nrows).style.apply(highlight_max, props=style.format(color=color), axis=0)

styled_df

Here we've made two additional reactive values (`style` and `color`), and written a Pandas pipeline reacting to those values, using precisely the same syntax you would with a regular Pandas expression. Since `styled_df` is now a reactive Pandas expression, it will re-run whenever any of those changes. To see, try executing each of the following commands, one by one:

In [ ]:
color.rx.value = 'red'

In [ ]:
nrows.rx.value += 2

In [ ]:
color.rx.value = 'darkblue'

In the code above, we made reactive strings, numbers, and DataFrame expressions. You can also make functions reactive, which lets you make the URL reactive as well:

In [ ]:
url = rx(URL)
df = rx(pd.read_csv)(url)
df.head(2)

In [ ]:
url.rx.value = 'https://datasets.holoviz.org/gapminders/v1/gapminders.csv'

In [ ]:
url.rx.value = URL

In this case, `df` wraps the `read_csv` call generating the DataFrame, rather than a specific DataFrame instance. This demonstrates the flexibility of reactive expressions: you can write code as you usually would, but gain control over its reactivity.

While we've been updating `.rx.value` manually in this notebook, you could easily replace these literals with widgets from ipywidgets or [HoloViz Panel](https://panel.holoviz.org), enabling you to create user-interactive, reactive applications with minimal effort.

## How it works

So, how does reactive programming in Param actually work? The underlying mechanism leverage's Python's [operator overloading](https://www.geeksforgeeks.org/operator-overloading-in-python/), which allows us to redefine common operators like '`+`' to perform additional tasks. When you use these operators in reactive expressions, Param not only carries out the operation but also records it, establishing a dependency from the reactive variables involved to the resulting expression. This means that when a reactive variable is updated, Param automatically updates any expressions dependent on it.

For instance, if you set up a reactive expression `j = i + 1`, where `i` has been made reactive, any change to `i` will automatically trigger an update in `j`, eliminating the need for manual event handling:

In [ ]:
i = rx(1)
j = i + 1
print(f'i = {i.rx.value}')
print(f'j = {j.rx.value}')

In [ ]:
i.rx.value = 7
print(f'i = {i.rx.value}')
print(f'j = {j.rx.value}')

Without `rx()`, adding 1 to `i` would have immediately invoked integer addition in Python, assigning an integer 2 to `j`. However, because we made `i` a reactive expression, what happens is that `i` stores its input value on an internal attribute called `_obj`, while overloading `+` to not just calculate `i + 1`, but also return another reactive object that records the operation (`i + 1`). This stores the dependency so that whenever `i` changes, the reactive object knows that it needs to update itself by re-executing `i + 1`.

In [ ]:
type(i), i._obj, j._operation

When you access the `.value` attribute of `j`, it retrieves the most recent result of the expression `i + 1`, automatically reapplying the operation if `i` has changed. For more complicated scenarios, reactive expressions can chain multiple operations and method calls together, executing them in sequence to obtain the final outcome.

In essence, reactive expressions are specialized Python objects that wrap standard objects. They record the operations you apply, and when an underlying reactive value changes, they automatically re-execute these operations. This eliminates the need for manually tracking and updating dependent variables, making it easier to build dynamic, responsive applications.

## Limitations

So does Python really allow _all_ operations to be overloaded so that a reactive expression works precisely like the underlying objects? 

Nearly, but not quite. For technical reasons, certain operations cannot be implemented in this way:

- Python requires the `len` operation to return an integer, not a deferred reactive integer
- The Python `is` statement always checks the immediate identity of its two operands, so it cannot be deferred  reactively
- Logical operators like `and`, `or`, `not`, and `in` are required to return Boolean types rather than deferred, reactive Boolean types
- No overloading is available for control flow keywords like `if`, `elif`, and `else` or ternary conditional expressions (i.e. `a if condition else b`), and so those actions cannot be captured for later reactive execution
- Iteration keywords like `for` or `while` can only be overloaded to some extent, specifically for fixed-length collections; other types of iteration cannot be captured for later reactive execution

However, Param's reactive expressions offer workarounds for these limitations through special methods under the `.rx` namespace to avoid confusion with the underlying object's own methods. We'll cover these methods in the next section.

## Special Methods on `.rx`

To circumvent the limitations explained above, the `.rx` namespace provides reactive versions of the operations that can't be made reactive through overloading:

- `.rx.in_()`: Reactive version of `in`, testing if the value is in the provided collection.
- `.rx.is_()`: Reactive version of `is`, testing the object identity against another object.
- `.rx.is_not()`: Reactive version of `is not`, testing the absence of object identity with another object.
- `.rx.len()`: Reactive version of `len()`, returning the length of the expression
- `.rx.pipe()`: Applies the given function (with static or reactive arguments) to this object.
- `.rx.when()`: Generates a new expression that only updates when the provided dependency updates.
- `.rx.where()`: Returns either the first or the second argument, depending on the current value of the expression.

Unlike their corresponding standard Python equivalent, each of these returns a reactive expression that can thus be combined with other reactive expressions to make reactive pipelines.

#### `.rx.in_(arg)`

Reactively checks if the current value is `.in_` the other collection

In [ ]:
rx(2).rx.in_([1, 2, 3])

#### `.rx.is_(arg)`

Reactively checks if the identity of the current value is the same as the argument to `.is_`

In [ ]:
rx(None).rx.is_(None)

#### `.rx.is_not(arg)`

Reactively checks if the identity of the current value is not the same as the argument to `.is_not`

In [ ]:
rx(None).rx.is_not(None)

#### `.rx.len()`

Returns the length of the object as a reactive expression

In [ ]:
obj = rx([1, 2, 3])
obj.rx.len()

In [ ]:
type(obj.rx.len())

#### `.rx.pipe(func, *args, **kwargs)`

Pipes the current value into a function as the first argument, passing in additional positional and keyword arguments if provided, and returning a reactive expression to replay that call as needed:

In [ ]:
def f(a, b): return a + b
    
rx(1).rx.pipe(f, 2)

`.rx.pipe` can be used with any Python function. One common usage is for making type conversion functions reactive:

In [ ]:
rx(8.5).rx.pipe(int)

In [ ]:
rx(8.5).rx.pipe(str)

#### `.rx.when(*conditions)`

Useful when creating UIs to declare that the expression should only update when some other parameter changes, e.g. when a user clicks a button or triggers an expensive operation through some other mechanism.

For instance, let's say we have some expensive function (here simulated using `time.sleep`). First, we bind parameters `a` and `b` to this function and create a reactive expression from this function.

In [ ]:
import time

def expensive_function(a, b):
    print(f'multiplying {a=} and {b=}')
    time.sleep(2)
    return a * b

a = rx(1)
b = rx(2)

expensive_expr = rx(expensive_function)(a, b)

The problem we face is that if we use this `expensive_expr` whenever `a` **or** `b` are changed, then the expensive computation gets triggered *twice* if we want to change both `a` _and_ `b`. To avoid unnecessary expense, we can gate the computation behind a third variable we'll name `gate`:

In [ ]:
gate = rx(False)
gated_expr = expensive_expr.rx.when(gate)

gated_expr

We can now safely change variables `a` and `b` separately without triggering the computation:

In [ ]:
a.rx.value = 2
b.rx.value = 4

gated_expr.rx.value

But when we trigger the `run` parameter the expression will re-compute:

In [ ]:
gate.rx.value=True

gated_expr.rx.value

#### `.rx.where(x, y)`

Reactive ternary conditional. In non-reactive Python code you can write:

```python
a if condition else b
``` 

to return value `a` or value `b` depending on some `condition`. However, Python does not allow overriding `if` to have special behavior for a reactive `condition`, and thus such an expression will immediately evaluate and return `a` or `b` rather than capturing this logic for later reactivity.

So if we want to have a reactive conditional, we have to rewrite the expression using `where`. First, we will declare a reactive `condition` expression to wrap a Boolean value that we can change later:

In [ ]:
condition = rx(True)

Now let's say we want to return either `a` or `b` depending on whether the `condition` is True or False. We can simply pass the values to `.where()`:

In [ ]:
a = rx(1)
b = rx(2)

ternary_expr = condition.rx.where(a, b)
ternary_expr

Since the initial value is `True` it returns the current value of `a`, which is `1`. However when we set the value to `False` it will return the value of `b`:


In [ ]:
condition.rx.value = False

ternary_expr.rx.value

Importantly, if we now change `b` the result will be reflected by the expression, reactively unless we explicitly resolve the result:

In [ ]:
b.rx.value = 5

ternary_expr.rx.value

Here the expression value depends only on `b` thanks to the `where` condition, and thus changes to `a` will no longer trigger any downstream updates until the condition is reversed again.

## Parameters and `param.bind`

Reactive expressions are part of the [Param](https://param.holoviz.org) library, and behind the scenes, all the reactivity is implemented using Parameters and their [dependencies and watchers](https://param.holoviz.org/user_guide/Dependencies_and_Watchers.html) support. You can use reactive expressions without needing to learn about Parameters, but if you do use Parameters in your work, they interact seamlessly with reactive expressions, providing a powerful and convenient way to organize your code and your work. In this section we will show how to use Parameters and the `param.bind` function together with reactive expressions for a more structured approach to reactive programming.

First, let's create a Parameterized class with a couple of Parameters:

In [ ]:
class Parameters(param.Parameterized): 
    a = param.Number(1)

    b = param.Number(0)

    run = param.Event()
    
p = Parameters()

Any of the parameters can be used as reactive expressions by calling `.rx()` on their Parameter object:

In [ ]:
expr = p.param.a.rx() + p.param.b.rx() + 3
expr

Now if we update the Parameter, the result of the expression will update immediately:

In [ ]:
p.b = 5
print(expr.rx.value)

You can thus use any Parameter in your reactive expressions, including Parameters from [HoloViz Panel](https://panel.holoviz.org) widgets.

### Binding Parameters to Functions

While reactive expressions with `rx` offer a flexible way to define dynamic relationships between Parameters, they operate at a fairly abstract level, encapsulating the underlying transformations. This can sometimes make it challenging to isolate specific parts of a pipeline for debugging or performance optimization.

Enter `param.bind`, which allows you to define functions that are automatically invoked when their input Parameters change. This serves as a bridge between the reactive `rx` model and the lower-level 'push' model. Unlike the 'push' model, where you would explicitly set up watchers and callbacks, `param.bind` simplifies the process by letting Param manage the mechanics, but also making the dependencies more transparent than in a purely `rx` approach.

In essence, `param.bind` offers the declarative nature of reactive expressions and the explicitness of the 'push' model. This makes it particularly useful for complex applications where you might want the clarity of explicit function calls for key parts of your pipeline, but also wish to retain the high-level, declarative relationships offered by reactive expressions.

To demonstrate this concept, let's define a simple Python function for adding numbers. We'll also include print statements to make it evident when the function is invoked:

In [ ]:
def add(a, b):
    print(f'add: {a}+{b}={a+b}')
    return a + b

add(3, 7)

Now we can use `param.bind` to "bind" parameters `a` and `b` to the `add` function's arguments to create a reactive function:

In [ ]:
reactive_add = param.bind(add, p.param.a, p.param.b)

reactive_add

As you can see, `reactive_add` works just like `add`, in that it adds two arguments, but in this case, it's taking the value of the `a` and `b` Parameters of `p`. Parameter `a` has been "bound" to the first argument and `b` to the second, and if either of them changes, the result changes. So if we change `p.a` to 5, the output above reacts immediately.

In [ ]:
p.a += 4

We can also call the reactive function explicitly to return the current result as a concrete, no longer reactive value:

In [ ]:
reactive_add()

The difference between `reactive_add` and `reactive_add()` is that the first one is a function, whose display will automatically update in IPython/Jupyter, while the second is a specific number (the result of calling that function a single time, never to be updated further):

In [ ]:
print(type(reactive_add), type(reactive_add()))

`param.bind` follows the semantics of Python's `functools.partial`, and so if you only partially bind the required arguments, you'll get a function of the remaining arguments:

In [ ]:
add_b = param.bind(add, p.param.a)
add_b

In [ ]:
add_b(5)

Note that you can bind any accepted type to make a reactive function, not just Parameters, but static values won't trigger reactive updates (here 38 will always be the same value, while the result will depend on the current value of `p.param.a`).

In [ ]:
param.bind(add, p.param.a, b=38)

Bound functions update their outputs reactively when displayed, but what if you want to use one in a reactive expression? You can easily do that if you call `.rx()` on a fully bound function to get a reactive expression to work with:

In [ ]:
param.bind(add, p.param.a, p.param.b).rx() / 2

As you can see, you can use bound functions to get reactivity if you prefer to write specific functions, or you can use reactive expressions to capture computations without writing function definitions, or you can combine the two as needed. Feel free to use the approach that best meets your needs!

And overall, hopefully, you can see that Param's reactive support provides a natural and powerful way to capture your computations in a way that can be replayed automatically whenever inputs change, making it a convenient basis for building interactive applications and computations.